In [1]:
import calendar
import xmlrpc.client
import csv
import pandas as pd
import phonenumbers
import psycopg2

ODOO_URL = "https://smsperkasa.odoo.com"
ODOO_DB = "smsperkasa-master-1574977"
ODOO_USERNAME = "admin"
ODOO_PASSWORD = "q82fD^YMXL246cUNr2Qp{a{TN"

def search_from_odoo(model, payload, fields, order=''):
        """Mixin function to search data from Odoo with Odoo's xmlrpc"""
        try:
            common = xmlrpc.client.ServerProxy(f'{ODOO_URL}/xmlrpc/2/common')
            dbname = ODOO_DB
            user = ODOO_USERNAME
            pw = ODOO_PASSWORD

            uid = common.authenticate(dbname, user, pw, {})

            models = xmlrpc.client.ServerProxy(f'{ODOO_URL}/xmlrpc/2/object')
            
            result = models.execute_kw(
                dbname,uid,pw,
                model,'search_read',[payload],
                {
                    'fields': fields, 
                    'context': {'lang': 'en_GB'}, 
                    'order': order
                }
            )

            return result
        except Exception as err:
            return ["Error", err]
        
# results_contact = search_from_odoo(
#     'res.partner',
#     [
#          ['email', 'not like', 'smsperkasa']
#     ],
#     ['name', 'phone', 'email', 'create_date', 'become_lead_date', 'become_mql_date', 'become_sql_date', 'become_customer_date', 'lifecycle_stage']
# )
# results_cust_only = search_from_odoo(
#     'res.partner',
#     [
#          ['email', 'not like', 'smsperkasa'],
#          ['lifecycle_stage', '=', 'customer']
#     ],
#     ['name', 'phone', 'email', 'create_date', 'become_lead_date', 'become_mql_date', 'become_sql_date', 'become_customer_date', 'lifecycle_stage']
# )
# results_sql_only = search_from_odoo(
#     'res.partner',
#     [
#          ['email', 'not like', 'smsperkasa'],
#          ['lifecycle_stage', '=', 'sql']
#     ],
#     ['name', 'phone', 'email', 'create_date', 'become_lead_date', 'become_mql_date', 'become_sql_date', 'become_customer_date', 'lifecycle_stage']
# )

# contact_ids = [ctc['id'] for ctc in results_contact]


# results_lead = search_from_odoo(
#     'crm.lead',
#     [
#         '|', ['active','=',True], ['active','=',False],
#         ['partner_id','not in', contact_ids]
#     ],
#     ['name', 'phone', 'email_from', 'create_date']
# )

In [132]:
def convert_phonenumber(phonenumber):
    """Function to convert phone number to the right format"""
    # if phonenumber[0] == '0':
    #     phonenumber = '+62' + phonenumber[1:]
    # parsed_phone = phonenumbers.parse(phonenumber, 'ID')
    # return phonenumbers.format_number(
    #     parsed_phone,
    #     phonenumbers.PhoneNumberFormat.INTERNATIONAL
    # )
    if pd.isna(phonenumber) == False:
        # phone = "+"+str(phonenumber).replace(".0", "")
        phone = phonenumber
        res = ''
        try:
            res = phonenumbers.format_number(phonenumbers.parse(phone.replace("+69","+62"),"ID"), phonenumbers.PhoneNumberFormat.NATIONAL).replace('0', '+62 ',1)
        except:
            # print(phone)
            res = phonenumbers.format_number(phonenumbers.parse(phone.replace("+","+62"),"ID"), phonenumbers.PhoneNumberFormat.NATIONAL).replace('0', '+62 ',1)
        return res
    else:
        return None


In [10]:
conn = psycopg2.connect(
    host="167.71.206.23",
    database="chatwoot_production",
    user="postgres",
    password="s0f!a_s3xy?!",
)

cursor = conn.cursor()
query = "select * from google_ads_conversions"

cursor.execute(query)

records = cursor.fetchall()
cursor.close()

In [15]:
conn = psycopg2.connect(
    host="167.71.206.23",
    database="chatwoot_production",
    user="postgres",
    password="s0f!a_s3xy?!",
)

In [16]:
cursor = conn.cursor()

In [17]:
query = """
SELECT * FROM contacts
WHERE
    (email IS NOT NULL AND email <> '') OR
    (phone_number IS NOT NULL AND phone_number <> '')
ORDER BY id
"""
cursor.execute(query)

In [18]:
records = cursor.fetchall()
cursor.close()

In [19]:
df = pd.DataFrame(records, columns=['id', 'name', 'email', 'phone_number', 'account_id', 'created_at', 'updated_at', 'additional_attributes', 'identifier', 'custom_attributes', 'last_activity_at'])
df

,id,name,email,phone_number,account_id,created_at,updated_at,additional_attributes,identifier,custom_attributes,last_activity_at
0,1,+62 821-3264-5835,None,+6282132645835,1,2022-02-19 12:00:31.290447,2022-03-08 04:58:39.532715,{},None,{},2022-03-08 04:58:39.523599
1,2,+62 813-2838-3224,None,+6281328383224,1,2022-02-19 12:00:45.584336,2022-03-22 10:00:53.459854,{},None,{},2022-03-22 10:00:53.454384
2,3,Teddy Wijaya,teddy@smsperkasa.com,+62315345215,1,2022-02-19 12:01:08.227124,2023-08-23 11:42:54.351529,"{'description': '', 'company_name': '', 'socia...",None,{},2023-08-23 11:42:54.343013
3,6,+62 823-2469-0268,None,+6282324690268,1,2022-02-19 12:12:29.387723,2022-02-19 12:12:29.492474,{},None,{},2022-02-19 12:12:29.487251
4,69,Renata Isnawati,None,+6283838073159,1,2022-02-19 14:36:51.014575,2022-02-21 02:29:22.151398,"{'description': '', 'company_name': '', 'socia...",None,{},2022-02-19 14:36:51.110114
...,...,...,...,...,...,...,...,...,...,...,...
20416,634543,Andika Angkat,andikaangkat098@gmail.com,+6281286286783,1,2023-10-30 03:49:45.230454,2023-10-30 03:49:45.230454,{},None,{},NaT
20417,634544,Johan,gabehouse2020@gmail.com,+6285299993356,1,2023-10-30 04:28:57.056763,2023-10-30 04:28:57.056763,{},None,{},NaT
20418,634545,Johan,None,+6285299993354,1,2023-10-30 04:29:11.589119,2023-10-30 04:44:55.099118,"{'city': '', 'country': '', 'description': 'Gr...",None,{},2023-10-30 04:44:55.095079
20419,634546,Fanny,None,+628124450088,1,2023-10-30 04:50:31.585651,2023-10-30 05:10:57.338340,"{'city': '', 'country': '', 'description': '',...",None,{},2023-10-30 05:10:57.331930


In [118]:
df_cw = pd.read_csv('data/chatwoot_contacts.csv')
df_cw['created_at_date'] = pd.to_datetime(df_cw['created_at_date'], errors='coerce')
df_cw.groupby([df_cw['created_at_date'].dt.year.rename('year'), df_cw['created_at_date'].dt.month.rename('month')]).agg({'count'})['created_at_date']
cleaned_cw_df = df_cw[(~df_cw['email'].isna() | ~df_cw['phone_number'].isna()) & ~df_cw['email'].str.contains("smsperkasa", na=False)]

In [133]:
df_cw = df
df_cw['created_at'] = pd.to_datetime(df_cw['created_at'], errors='coerce')
df_cw = df_cw.replace('', None)
df_cw.groupby([df_cw['created_at'].dt.year.rename('year'), df_cw['created_at'].dt.month.rename('month')]).agg({'count'})['created_at']
cleaned_cw_df = df_cw[(~df_cw['email'].isna() | ~df_cw['phone_number'].isna()) & ~df_cw['email'].str.contains("smsperkasa", na=False)]

In [134]:
cw_emails = cleaned_cw_df['email'].values
cw_phones = cleaned_cw_df['phone_number'].values

In [135]:
cw_emails = cw_emails[~pd.isna(cw_emails)]
cw_phones = cw_phones[~pd.isna(cw_phones)]
# phones = ["+"+str(phone).replace(".0", "") for phone in cw_phones]
correct_phones = []
for phone in cw_phones:
    # print(phone)
    try:
        correct_phones.append(
            phonenumbers.format_number(phonenumbers.parse(phone.replace("+69","+62"),"ID"), phonenumbers.PhoneNumberFormat.NATIONAL).replace('0', '+62 ',1)
        )
    except:
        correct_phones.append(
            phonenumbers.format_number(phonenumbers.parse(phone.replace("+","+62"),"ID"), phonenumbers.PhoneNumberFormat.NATIONAL).replace('0', '+62 ',1)
        )
# correct_phones = [phonenumbers.format_number(phonenumbers.parse(nomor,"ID"), phonenumbers.PhoneNumberFormat.NATIONAL).replace('0', '+62 ',1) for nomor in phones]
# phonenumbers.format_number(phonenumbers.parse(phones[0]), phonenumbers.PhoneNumberFormat.NATIONAL).replace('0', '+62 ',1)
# correct_phones

In [136]:
odoo_contacts = search_from_odoo(
    'res.partner',
    [
        # '|',
        ['email', 'not in', list(cw_emails)],
        ['phone', 'not in', list(correct_phones)],
        ['email', 'not like', 'smsperkasa'],
        # ['lifecycle_stage', 'in', ['leads', 'mql']]
    ],
    ['name', 'phone', 'email', 'create_date']
)
df_contacts = pd.DataFrame(odoo_contacts)
odoo_emails = df_contacts[df_contacts.email != False]['email'].values
odoo_phones = df_contacts[df_contacts.phone != False]['phone'].values

In [137]:
results_leads = search_from_odoo(
    'crm.lead',
    [
        "&", "|", ["active", "=", True], ["active", "=", False], ["type", "=", "lead"]
    ],
    ['name', 'phone', 'email_from', 'create_date', 'type']
)
df_leads = pd.DataFrame.from_dict(results_leads)

In [139]:
df_leads = df_leads[(~df_leads.email_from.isin(cw_emails)) & (~df_leads.phone.isin(correct_phones)) & (~df_leads.email_from.isin(odoo_emails)) & (~df_leads.phone.isin(odoo_phones))]
df_leads = df_leads[['name', 'email_from', 'phone', 'create_date']]
df_leads.rename(columns={
    'email_from': 'email'
}, inplace=True)

In [107]:
cleaned_cw_df[cleaned_cw_df.create_date < '2023-10-02'].sort_values(by='create_date', ascending=False)

,name,email,phone,create_date
18846,+62 813-3022-7699,None,+62 813-3022-7699,2023-09-30 12:43:49.904456
18845,Haqi,heqiul@yahoo.co.id,+62 858-6525-3699,2023-09-30 09:13:44.502978
18844,Hery,heryadyrustam@gmail.com,+62 853-2222-2862,2023-09-30 08:47:13.779062
18843,+62 823-1317-5207,None,+62 823-1317-5207,2023-09-30 08:44:29.986338
18842,Nasrul Munaf,azasnafs2@gmail.com,+62 853-2533-4199,2023-09-30 08:43:55.229400
...,...,...,...,...
5,Adrian,None,+62 856-4002-5698,2022-02-19 16:53:03.380937
4,Renata Isnawati,None,+62 838-3807-3159,2022-02-19 14:36:51.014575
3,+62 823-2469-0268,None,+62 823-2469-0268,2022-02-19 12:12:29.387723
1,+62 813-2838-3224,None,+62 813-2838-3224,2022-02-19 12:00:45.584336


In [140]:
df_leads[df_leads.create_date < '2023-10-02'].sort_values(by='create_date', ascending=False)

,name,email,phone,create_date
2539,"Pameran HOMEDEC 2023, Dibuka Sampai Besok!","""HOMEDEC Indonesia"" <info@homedec.co.id>",False,2023-09-30 14:38:55
2571,HOMEDEC 2023 Your #OneStopHomeSolution 🏡✨,"""HOMEDEC Indonesia"" <info@homedec.co.id>",False,2023-09-30 03:48:54
2601,AI-mazing September: Protecting AI Technologie...,"""[KASS] ASEAN IP Updates - September 2023 issu...",False,2023-09-29 07:13:59
2609,Permintaan Penawaran Harga No.: 2107-1,"""Dewi Setyawati"" <dewisimsby@gmail.com>",False,2023-09-29 05:03:51
2611,WIRE MESH SS 316 STANDARD JIS // SMS PERKASA,"""Sunan"" <sunan@pttomatec.com>",False,2023-09-29 04:33:53
...,...,...,...,...
20375,Puri Suma Mandiri,False,False,2022-01-03 07:30:56
20376,ASA Group,False,False,2022-01-03 05:03:23
20378,Indra Jaya Group,False,False,2022-01-03 04:31:24
20379,Universitas terbuka di merr,False,+62 274 513976,2022-01-03 04:28:25


In [130]:
df_contacts[df_contacts.create_date < '2023-10-02'].sort_values(by='create_date', ascending=False)

,id,name,phone,email,create_date
1904,61117,Batang Alum Industrie,+62 857-4249-7571,herdianpebi85@gmail.com,2023-09-30 06:10:29
1906,61118,herdian,+62 857-4249-7571,herdianpebi85@gmail.com,2023-09-30 06:10:29
10083,61116,Imma,+62 812-3398-0785,anis.sms@gmail.com,2023-09-30 04:25:12
10082,61115,Sabar Makmur Barokah,False,False,2023-09-30 04:24:38
6116,61114,Sigit,+62 813-2555-8899,Sigit_scalper@yahoo.co.id,2023-09-30 03:59:08
...,...,...,...,...,...
6473,29193,Khairil,+62 811-330-517,pt.bkpratama@yahoo.com,2021-12-29 07:41:57
333,10,Administrator,False,admin@example.com,2021-12-15 04:04:38
8723,9,Pak B,False,False,2020-10-16 08:33:33
8720,8,Pak A,False,False,2020-10-16 08:25:47


In [129]:
mql_df[mql_df.create_date < '2023-10-02'].sort_values(by='create_date', ascending=False)

,name,email,phone,create_date
14753,Arief Hadiansyah Tri Ps,hadiansyaharief22@gmail.com,+62 896-6227-1906,2023-10-01 23:32:14
14754,Dedy Kuswanto,dedy3547@gmail.com,+62 852-3816-6803,2023-10-01 23:32:14
14755,Hadi suwito,witodimas31@gmail.com,+62 858-8162-2746,2023-10-01 23:26:57
14756,Supriyadi 2023,supriyadi202305@gmail.com,+62 857-1680-4560,2023-10-01 23:09:02
14757,Teguh Edi,indonesia.sarapan@gmail.com,+62 857-6001-0966,2023-10-01 00:19:05
...,...,...,...,...
11643,"Sylvia, Ny. Jd",False,081357757777,2021-12-29 07:41:57
11638,Syamsuri,False,+62 812-7869-5149,2021-12-29 07:41:57
6251,Karya Indah Medika,puji.pmbkim@gmail.com,+62 857-4909-7474,2021-12-29 07:41:57
8720,Pak A,False,False,2020-10-16 08:25:47


In [141]:
cleaned_cw_df['phone'] = cleaned_cw_df.apply(lambda x: convert_phonenumber(x.phone_number), axis=1)

cleaned_cw_df = cleaned_cw_df[['name', 'email', 'phone', 'created_at']]
cleaned_cw_df.rename(columns={
    'created_at': 'create_date'
}, inplace=True)

/var/folders/w3/b14k9y695c912pkmbdrg1k900000gn/T/ipykernel_3170/2356783971.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_cw_df['phone'] = cleaned_cw_df.apply(lambda x: convert_phonenumber(x.phone_number), axis=1)
/var/folders/w3/b14k9y695c912pkmbdrg1k900000gn/T/ipykernel_3170/2356783971.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_cw_df.rename(columns={


In [142]:
mql_df = df_contacts[['name', 'email', 'phone', 'create_date']].append(df_leads, ignore_index = True)
mql_df = mql_df.append(cleaned_cw_df, ignore_index = True)
mql_df['create_date'] = pd.to_datetime(mql_df['create_date'], errors='coerce')
mql_df = mql_df.sort_values(by=['create_date'])
mql_df = mql_df.drop_duplicates(subset=['email', 'phone'])

/var/folders/w3/b14k9y695c912pkmbdrg1k900000gn/T/ipykernel_3170/431163569.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  mql_df = df_contacts[['name', 'email', 'phone', 'create_date']].append(df_leads, ignore_index = True)
/var/folders/w3/b14k9y695c912pkmbdrg1k900000gn/T/ipykernel_3170/431163569.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  mql_df = mql_df.append(cleaned_cw_df, ignore_index = True)


In [ ]:
# mql_df[mql_df.isin(mql_df[mql_df.duplicated(subset=['email', 'phone'])])]

Summary

In [143]:
date_range_2023 = mql_df[(mql_df['create_date'] >= '2023-10-01') & (mql_df['create_date'] <= '2023-12-31')]
date_groupby_mql_2023 = date_range_2023.groupby([date_range_2023['create_date'].dt.date.rename('month')]).agg({'count'})['create_date']

date_range_2022 = mql_df[(mql_df['create_date'] >= '2022-01-01') & (mql_df['create_date'] <= '2022-12-31')]
date_groupby_mql_2022 = date_range_2022.groupby([date_range_2022['create_date'].dt.month.rename('month')]).agg({'count'})['create_date']

In [144]:
date_groupby_mql_2022

,count
month,
1,1988
2,403
3,2338
4,598
5,525
6,858
7,650
8,885
9,774


In [158]:
date_groupby_mql_2023

,count
month,
2023-10-02,91
2023-10-03,65
2023-10-04,79
2023-10-05,75
2023-10-06,103
2023-10-07,49
2023-10-08,2
2023-10-09,99
2023-10-10,71


In [153]:
mql_df[mql_df.phone == '+62 858-8646-3596']

,name,email,phone,create_date
35609,Che Che Cutdini,None,+62 858-8646-3596,2023-10-05 03:11:09.146088


In [159]:
xx = mql_df[(mql_df['create_date'] >= '2023-10-02') & (mql_df['create_date'] <= '2023-10-03')].sort_values(by='create_date')

In [160]:
xx

,name,email,phone,create_date
35354,Yoseflina,yosefilinasampe@gmail.com,+62 813-3966-3792,2023-10-02 00:43:26.006377
35355,dito Saputra,None,+62 856-0414-5024,2023-10-02 00:51:59.226937
35356,suhardi jaya,suhardijaya44@gmail.com,+62 853-4293-0003,2023-10-02 00:54:42.913626
35357,+62 812-4200-0174,None,+62 812-4200-0174,2023-10-02 01:06:16.751314
35358,Indra,None,+62 822-6669-9422,2023-10-02 01:18:03.595424
...,...,...,...,...
35438,vinkan,kimvinkan@gmail.com,+62 851-5509-9273,2023-10-02 09:37:35.483754
12745,Kepala Mumet karena Kabel Jaringan Ruwet? Patc...,"""Aurelia - XDC"" <newsletter@xdc-indonesia.com>",False,2023-10-02 10:04:01.000000
35439,Winny,None,+62 818-893-020,2023-10-02 10:21:49.758315
12744,Pelatihan Nasional Dua Hari dengan tema : “PPI...,"""bumn pelatihan"" <bumn.pelatihan02@gmail.com>",False,2023-10-02 15:18:58.000000


In [161]:
with pd.ExcelWriter("/Users/bcw-003/Downloads/test_1.xlsx") as writer:
    xx.to_excel(writer, sheet_name='contacts', index=False)

In [169]:
df_contacts[df_contacts.email == 'ismedst@gmail.com']

,id,name,phone,email,create_date


In [170]:
df_leads[df_leads.email == 'ismedst@gmail.com']

,name,email,phone,create_date


In [166]:
cleaned_cw_df[cleaned_cw_df.email == 'ismedst@gmail.com']

,name,email,phone,create_date
18853,Ismed S,ismedst@gmail.com,+62 852-8940-4589,2023-10-02 01:48:37.728082
